In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
from collections import Counter
import math
import scipy.stats as st
import seaborn as sns

# Init Variables

In [ ]:
NODE_NUMBER = 100
MAX_EDGE_NUMBER = (NODE_NUMBER) * (NODE_NUMBER - 1) / 2
# er parameters:
UPPER_EDGE_PROBABILITY = 0.06
LOWER_EDGE_PROBABILITY = 0.02
EDGE_PROBABILITY = 0.05
EDGE_NUMBER = int(MAX_EDGE_NUMBER * EDGE_PROBABILITY)  # Approximately 0.05 of max possible edge number
# rsrg parameters:
UPPER_COPY_NUMBER_D1 = 15
LOWER_COPY_NUMBER_D1 = 10
LOWER_RSRG_P = 0.125
LOWER_AVERAGE_D_RSRG = 2
UPPER_AVERAGE_D_RSRG = 6
RSRG_PROBABILITIES = [1, 0.75, 0.5, 0.25, 0.125]
#rsrbg parameters:
VALID_D_PAIRS_FOR_NODE_NUMBER = []
for x in range(2, int(NODE_NUMBER / 4)):
    y = x
    while y < (NODE_NUMBER / 4):
        if NODE_NUMBER % (x + y) == 0:
            VALID_D_PAIRS_FOR_NODE_NUMBER.append((x, y))
        y = y + 1

GRAPH_NUMBER = 100
plt.figure(figsize = (8,8)) #Specify the size of figure
SIMULATION_NUMBER = 100
COLOR = ['red', 'green', 'blue', 'gray', 'yellow', 'brown', 'black']

# Functions

### Eigvals

###### Adjacency

In [ ]:
def get_adjacency_eigvals(graph):
    L = nx.adjacency_matrix(graph)
    e = np.linalg.eigvals(L.toarray())
    e = list(sorted(e))
    return e

###### Laplacian

In [ ]:
def get_laplacian_evigal(graph): # Laplacian matrix
    L = nx.laplacian_matrix(graph)
    e = np.linalg.eigvals(L.toarray())
    e = list(sorted(e))
    return e

### Spectral Gap

In [ ]:
def get_spectral_gap(graph):
    eigvals = get_adjacency_eigvals(graph)
    max_index = len(eigvals) - 1
    maximum = eigvals[max_index]
    second_max = eigvals[max_index - 1]
    diff = maximum - second_max
    return diff

### Algebraic Connectivity

In [ ]:
def get_algebraic_connectivity(graph):
    return nx.algebraic_connectivity(graph)

### check for connection with Algebraic Connectivity

In [ ]:
def is_graph_connected(graph):
    return get_algebraic_connectivity(graph) > 0

### Trace Power S

In [ ]:
def get_trace_power_s(graph, power=2): # number of walks with len s (power) in graph
    eigvals = get_adjacency_eigvals(graph)
    eigvals_powers = np.power(eigvals, power)
    summation = np.sum(eigvals_powers)
    return summation

### Phi s (Average Trace Power S)

In [ ]:
def get_phi_s(graph): # average number of walks with len s (power) in graph
    eigvals = get_adjacency_eigvals(graph)
    eigvals_powers = np.power(eigvals, power)
    summation = np.sum(eigvals_powers)
    avg = summation / len(eigvals)
    return summation

### Centrality of Global Subgraph

In [ ]:
def get_centrality_of_global_subgraph(graph):
    eigvals = get_adjacency_eigvals(graph)
    summation = sum([math.exp(value) for value in eigvals])
    return summation

### Average Eigvals

In [ ]:
def get_average_eigvals(graph):
    summation = get_centrality_of_global_subgraph(graph)
    ln = math.log(summation)
    return ln

## Automorphism

In [ ]:
def get_all_automorphism(graph): # automorphism is isomorphism for a graph with itself
    dictionary = nx.vf2pp_all_isomorphisms(graph, graph)
    return list(dictionary) # return a list of mapping (return a list of dictionary)

### Node Similarity (Vertex Transitivity)

In [ ]:
def is_node_similar(graph):
    automorphisms = get_all_automorphism(graph)
    for u,v in graph.edges:
        if not any(auto[u] == v for auto in automorphisms):
            return False
    return True

### Symmetry (Edge Transitivity)

In [ ]:
def is_symmetry(graph):
    automorphisms = get_all_automorphism(graph)
    for u, v in graph.edges:
        for x, y in graph.edges:
            if not any(auto[u] == x and auto[v] ==y for auto in automorphisms):
                return False
    return True

### Laplacian Energy

In [ ]:
def get_laplacian_energy(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_adjacency_eigvals(graph)
    laplacian_energy = sum([abs(value) for value in eigenvalues])
    return laplacian_energy

In [ ]:
def get_laplacian_energy2(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_adjacency_eigvals(graph)
    summation = sum([value if value > 0 else 0 for value in eigenvalues])
    laplacian_energy = summation * 2
    return laplacian_energy

In [ ]:
def get_laplacian_energy3(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_laplacian_evigal(graph)
    n = graph.number_of_nodes()
    m = graph.number_of_edges()
    constant = (2 * m) / n
    laplacian_energy = sum([abs(value - constant) for value in eigenvalues])
    return laplacian_energy

### Plotting Graphs

In [ ]:
def show_graph(graph, path=None, labels=False):
    pos = nx.circular_layout(graph)
    plt.figure(figsize = (12, 12))
    nx.draw_networkx(graph, pos, with_labels=labels)
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Degree Distribution

In [ ]:
def degree_distribution(graph, path=None, style='-o'):
    degrees = [graph.degree(n) for n in graph.nodes()]
    degrees = list(sorted(degrees))
    degree_freq_dic = Counter(degrees)
    x_axis = degree_freq_dic.keys()
    y_axis = degree_freq_dic.values()
    y_axis = np.array(list(y_axis)) / len(degrees)
    
    plt.title('Degree Distribution')
    plt.xlabel("Degree")
    plt.ylabel("Frequesncy")
    plt.plot(x_axis, y_axis, style, label='degree probability')
    
    upper_y = np.array([0, max(y_axis)])
    avg = np.average(degrees)
    upper_x = np.array([avg, avg])
    plt.plot(upper_x, upper_y, color='red', linestyle='-.', label='mean')
    plt.legend(loc='best') # setting best location for labels
    
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Double-Log

In [ ]:
def double_log(graph, path=None, style='-o'):
    degrees = [graph.degree(n) for n in graph.nodes()]
    degrees = list(sorted(degrees))
    degree_freq_dic = Counter(degrees)
    unique_degrees = list(degree_freq_dic.keys())
    frequency = list(degree_freq_dic.values())
    x_axis = np.log(unique_degrees)
    y_axis = np.log(frequency)
    y_axis = np.array(list(y_axis)) / len(degrees)
    plt.xlabel("Degree")
    plt.ylabel("Degree Distribution")
    plt.title('Double Log')
    plt.plot(x_axis, y_axis, style, label='degree distribution')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

## Comparing Plots

In [ ]:
def compare_datas(datas, labels, x_label='', y_label='', title='', style='-o', color=COLOR, path=None):
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    for i in range(len(datas)):
        x_axis = list(range(len(datas[i])))
        plt.plot(x_axis, datas[i], style, label=labels[i], color=color[i])
    if path != None:
        plt.savefig(path+'.png')
    plt.legend(loc='best')
    plt.show()

# PDF

In [ ]:
def draw_pdf(data, x_label='data number', y_label='value', title='pdf function', style='-o', color=COLOR, path=None):
    data = np.array(list(sorted(data)))
    mean = np.mean(data)
    std = np.std(data)
    pdf = st.norm.pdf(data, mean, std)
    
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.plot(data, pdf, style)
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def draw_pdf2(data, x_label='data number', y_label='value', title='pdf function', style='-o', color=COLOR, path=None):
    plt.figure(figsize = (5,5))
    sns.kdeplot(data , bw = 0.5 , fill = True)
    
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def draw_pdf3(data, x_label='data number', y_label='value', title='pdf function', style='-o', color=COLOR, path=None):
    sns.set_style("whitegrid")  # Setting style(Optional)
    sns.distplot(x=data, bins=10, kde=True, color='teal', kde_kws=dict(linewidth=4, color='green')) #kde for normal distribution
    
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

## Data Details

### c.i plots

In [ ]:
def coefficient_interval_plot(data, path=None, alpha=0.95):
    x = np.array([i for i in range(len(data))])
    y = np.array(data)
    # plotting
    plt.plot(y, x,'o', color='blue', label='data')
    
    # confidence intervals
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
#     print(f"\n{alpha*100} confidence interval {lower} and {upper}")
    
    # c.i upper & lower
    upper_y = np.array([0, len(data)])
    upper_x = np.array([upper, upper])
    plt.plot(upper_x, upper_y, color='red', linestyle='-.', label='upper c.i')
    
    lower_y = np.array([0, len(data)])
    lower_x = np.array([lower, lower])
    plt.plot(lower_x, lower_y, color='orange', linestyle='-.', label='lower c.i')
    
    ci_x = np.array([lower, upper])
    ci_y = np.array([0, 0])
    plt.plot(ci_x, ci_y, '-', color='green', label='c.i')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot2(data, path=None, alpha=0.95):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    # confidence intervals
    ci = (1.0-alpha) * np.std(y) / np.mean(y)
    mean = np.mean(y)
    avg = [mean for i in range(len(data))]
    
    # Plot the confidence interval
    plt.fill_between(x, (avg-ci), (avg+ci), color='blue', alpha=0.1)
    plt.plot(x, (avg-ci), '--', color='blue', label='-*ci')
    plt.plot(x, (avg+ci), '--', color='blue', label='+*ci')
    plt.fill_between(x, (avg-2*ci), (avg+2*ci), color='green', alpha=.1)
    plt.plot(x, (avg-2*ci), '--', color='green', label='-2*ci')
    plt.plot(x, (avg+2*ci), '--', color='green', label='+2*ci')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot3(data, path=None, alpha=0.95):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    # Define the confidence interval
    ci = (1.0-alpha) * np.std(y) / np.mean(y)
    
    # Plot the confidence interval
    plt.fill_between(x, (y-ci), (y+ci), color='blue', alpha=0.1)
    plt.plot(x, (y-2*ci), '--', color='blue', label='-*ci')
    plt.plot(x, (y+2*ci), '--', color='blue', label='+*ci')
    plt.fill_between(x, (y-2*ci), (y+2*ci), color='green', alpha=.1)
    plt.plot(x, (y-2*ci), '--', color='green', label='-2*ci')
    plt.plot(x, (y+2*ci), '--', color='green', label='+2*ci')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot4(data, path=None):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    y.astype('float64')
                 
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    low, high = st.norm.interval(alpha=0.9, loc=np.mean(data), scale=st.sem(data))
    low2, high2 = st.norm.interval(alpha=0.95, loc=np.mean(data), scale=st.sem(data))
    low3, high3 = st.norm.interval(alpha=0.99, loc=np.mean(data), scale=st.sem(data))
    
    # Plot the confidence interval
    plt.fill_between(x, [low3 for i in x], [high3 for i in x], color='green', alpha=0.1)
    plt.plot(x, [low3 for i in x], '--', color='green', label='alpha=0.99')
    plt.plot(x, [high3 for i in x], '--', color='green')
    
    plt.fill_between(x, [low2 for i in x], [high2 for i in x], color='orange', alpha=0.1)
    plt.plot(x, [low2 for i in x], '--', color='orange', label='0.95')
    plt.plot(x, [high2 for i in x], '--', color='orange')
    
    plt.fill_between(x, [low for i in x], [high for i in x], color='blue', alpha=0.1)
    plt.plot(x, [low for i in x], '--', color='blue', label='0.9')
    plt.plot(x, [high for i in x], '--', color='blue')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot5(data, path=None):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    y.astype('float64')
                 
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    # Plot the confidence interval
    alpha = 0.99
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
    
    plt.fill_between(x, [lower for i in x], [upper for i in x], color='green', alpha=0.1)
    plt.plot(x, [lower for i in x], '--', color='green', label='alpha=0.99')
    plt.plot(x, [upper for i in x], '--', color='green')
    
    alpha = 0.95
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
    
    plt.fill_between(x, [lower for i in x], [upper for i in x], color='orange', alpha=0.1)
    plt.plot(x, [lower for i in x], '--', color='orange', label='0.95')
    plt.plot(x, [upper for i in x], '--', color='orange')
    
    alpha = 0.9
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
    
    plt.fill_between(x, [lower for i in x], [upper for i in x], color='blue', alpha=0.1)
    plt.plot(x, [lower for i in x], '--', color='blue', label='0.9')
    plt.plot(x, [upper for i in x], '--', color='blue')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Mean & Variance

In [ ]:
def get_details(data):
    print('mean: ', np.mean(data))
    print('variance: ', np.var(data))
    print('std: ', np.std)

# Random Semi Regular Bipartite Graph (RSRBG)

In [ ]:
def rewire_edge_rsrbg(graph, node1, node2):
    for u, v in graph.edges:
        n1_node = min([u, v])
        n2_node = max([u, v])
        if n1_node != node1 and n2_node != node2 and (not graph.has_edge(n1_node, node2)) and (not graph.has_edge(node1, n2_node)):
            graph.remove_edge(u, v)
            graph.add_edge(n1_node, node2)
            graph.add_edge(node1, n2_node)
            break



def generate_rsrbg(d1, d2, n, seed=None):
    if (n * d2) % (d1 + d2) != 0:
        raise Exception(f"can't make graph with {n} nodes!")
        
    if seed != None:
        random.seed(seed)
    
    n1 = int((n * d2) / (d1 + d2))
    n2 = n - n1
    
    graph = nx.Graph()
    graph.add_nodes_from(list(range(n)))
    
    n1_stubs = list(range(n1)) * d1
    n2_stubs = list(range(n1, n)) * d2
    
    while len(n1_stubs) > 0:
        rnd1 = random.randint(0, len(n1_stubs) - 1)
        rnd2 = random.randint(0, len(n2_stubs) - 1)
        if graph.has_edge(n1_stubs[rnd1], n2_stubs[rnd2]):
            continue
        graph.add_edge(n1_stubs[rnd1], n2_stubs[rnd2])
        n1_stubs.pop(rnd1)
        n2_stubs.pop(rnd2)
        
        has_chance = False
        for s1 in n1_stubs:
            for s2 in n2_stubs:
                if not graph.has_edge(s1, s2):
                    has_chance = True
                    break
            if has_chance:
                break;
        
        if not has_chance: # we can't continue and add anymore edge
            for i in range(len(n1_stubs)):
                rewire_edge_rsrbg(graph, n1_stubs[i], n2_stubs[i])
            break
        
    return graph

# Random Semi Regular Graph(RSRG)

In [ ]:
def rewire_edge_rsrg(graph, node1, node2):
    for u, v in graph.edges:
        if not any(i in [u, v] for i in [node1, node2]): #node1 and node2 are completely different from u and v (no same value))
            if (not graph.has_edge(u, node1)) and (not graph.has_edge(node2, v)):
                graph.add_edge(u, node1)
                graph.add_edge(v, node2)
                break
            elif (not graph.has_edge(v, node1)) and (not graph.has_edge(node2, u)):
                graph.add_edge(v, node1)
                graph.add_edge(u, node2)
                break


def should_add_edge(p):
    if random.random() <= p:
        return True
    return False


def generate_rsrg(p, d1, d2, n, seed=None):
    if p < 0 or p > 1:
        raise Exception("p must be between 0 and 1")
    
    if seed != None:
        random.seed(seed)
    
    n1 = math.floor(n * (1 - p))
    n2 = n - n1
    
    n1_stubs = list(range(n1)) * d1
    n2_stubs = list(range(n1, n)) * d2
    stubs = n1_stubs + n2_stubs
    
    graph = nx.Graph()
    graph.add_nodes_from(list(range(n)))
    
    while len(stubs) > 1:
        if len(stubs) == 2:
            if graph.has_edge(stubs[0], stubs[1]) or stubs[0] == stubs[1]:
                if should_add_edge(p):
                    rewire_edge_rsrg(graph, stubs[0], stubs[1])
            else:
                if should_add_edge(p):
                    graph.add_edge(stubs[0], stubs[1])
            break
        else:
            rnd = random.randint(0, len(stubs) - 1)
            node1 = stubs[rnd]
            stubs.pop(rnd)
            rnd = random.randint(0, len(stubs) - 1)
            node2 = stubs[rnd]
            stubs.pop(rnd)
            if node1 == node2 or graph.has_edge(node1, node2):
                stubs.append(node1)
                stubs.append(node2)
            else:
                if should_add_edge(p):
                    graph.add_edge(node1, node2)
            
            has_chance = False
            for i in range(len(stubs)):
                j = i
                while(j < len(stubs)):
                    if stubs[i] != stubs[j] and (not graph.has_edge(stubs[i], stubs[j])):
                        has_chance = True
                    j = j + 1
                if has_chance: # we still can continue
                    break
            
            if not has_chance: # we can't continue and add anymore edge
                break
            
    
    return graph

# Making Graphs

### RSRBG

In [ ]:
seed_values = random.sample(range(1, 100000), GRAPH_NUMBER) # generating GRAPH_NUMBER unique random number to be used as seed
rsrbgs = []
for i in range(GRAPH_NUMBER):
    d1, d2 = random.choice(VALID_D_PAIRS_FOR_NODE_NUMBER)
    rsrbgs.append(generate_rsrbg(d1, d2, NODE_NUMBER))
    average_d = (2 * d1 * d2) / (d1 + d2)

In [ ]:
for graph in rsrbgs:
    show_graph(graph, labels=True)


In [ ]:
for graph in rsrbgs:
    degree_distribution(graph)

### RSRG

In [ ]:
seed_values = random.sample(range(1, 34098),GRAPH_NUMBER)

rsrgs = []
probability_of_rsrg = []
d1_of_rsrg = []
d2_of_rsrg = []
for i in range(GRAPH_NUMBER):
    average_d = 0
    p = 0
    d1 = 0
    d2 = 0
    n = 0
    while average_d < LOWER_AVERAGE_D_RSRG or average_d > UPPER_AVERAGE_D_RSRG or p < LOWER_RSRG_P:
        p = random.random()
        d1 = random.randint(LOWER_COPY_NUMBER_D1, UPPER_COPY_NUMBER_D1)
        d2 = random.randint(LOWER_COPY_NUMBER_D1, UPPER_COPY_NUMBER_D1)
        n = NODE_NUMBER
        average_d = (d1 * (1 - p) + p * d2) * p
    
    
    rsrgs.append(generate_rsrg(p, d1, d2, n, seed=seed_values[i]))
    print(f'p: {p}, d1: {d1}, d2: {d2}, <d>: {average_d}')
    probability_of_rsrg.append(p)
    d1_of_rsrg.append(d1)
    d2_of_rsrg.append(d2)

In [ ]:
for graph in rsrgs:
    show_graph(graph, labels=True)

In [ ]:
for i in range(len(rsrgs)):
    p = probability_of_rsrg[i]
    d1 = d1_of_rsrg[i]
    d2 = d2_of_rsrg[i]
    average_d = (d1 * (1 - p) + p * d2) * p
    print('<d>:', average_d)
    degree_distribution(rsrgs[i])

### ER

In [ ]:
er_seed_values = random.sample(range(1, 100000), GRAPH_NUMBER) # generating GRAPH_NUMBER unique random number to be used as seed

randoms = []
for i in range(GRAPH_NUMBER):
    print('graph no: ', i)
    graph = nx.erdos_renyi_graph(NODE_NUMBER, EDGE_PROBABILITY, seed=er_seed_values[i])
    randoms.append(graph)

In [ ]:
for graph in randoms:
    show_graph(graph, labels=True)

In [ ]:
for graph in randoms:
    degree_distribution(graph)

# Algebraic Connectivity

###### RSRBG

In [ ]:
rsrbg_ac = []
i = 0
for graph in rsrbgs:
    rsrbg_ac.append(get_algebraic_connectivity(graph))
    print(i)
    i = i + 1

get_details(rsrbg_ac)
coefficient_interval_plot(rsrbg_ac)
coefficient_interval_plot2(rsrbg_ac)
coefficient_interval_plot3(rsrbg_ac)
coefficient_interval_plot4(rsrbg_ac)
coefficient_interval_plot5(rsrbg_ac)

###### RSRG

In [ ]:
rsrg_ac = []
for graph in rsrgs:
    rsrg_ac.append(get_algebraic_connectivity(graph))

get_details(rsrg_ac)
coefficient_interval_plot(rsrg_ac)
coefficient_interval_plot2(rsrg_ac)
coefficient_interval_plot3(rsrg_ac)
coefficient_interval_plot4(rsrg_ac)
coefficient_interval_plot5(rsrg_ac)

###### ER

In [ ]:
random_ac = []
for graph in randoms:
    random_ac.append(get_algebraic_connectivity(graph))

get_details(random_ac)
coefficient_interval_plot(random_ac)
coefficient_interval_plot2(random_ac)
coefficient_interval_plot3(random_ac)
coefficient_interval_plot4(random_ac)
coefficient_interval_plot5(random_ac)

### Compare

In [ ]:
datas = [rsrbg_ac, rsrg_ac, random_ac]
labels = ['rsrbg', 'rsrg', 'er']
x_label = 'graph number'
y_label = 'algebraic connectivity'
title = 'comparing algebraic connectivity of graphs'
compare_datas(datas, labels, x_label=x_label, y_label=y_label, title=title)

# Spectral Gap

###### RSRBG

In [ ]:
rsrbg_sg = []
for graph in rsrbgs:
    rsrbg_sg.append(get_spectral_gap(graph))

get_details(rsrbg_sg)
coefficient_interval_plot(rsrbg_sg)
coefficient_interval_plot2(rsrbg_sg)
coefficient_interval_plot3(rsrbg_sg)
try:
    coefficient_interval_plot4(rsrbg_sg)
except:
    plt.show()
coefficient_interval_plot5(rsrbg_sg)

###### RSRG

In [ ]:
rsrg_sg = []
for graph in rsrgs:
    rsrg_sg.append(get_spectral_gap(graph))

get_details(rsrg_sg)
coefficient_interval_plot(rsrg_sg)
coefficient_interval_plot2(rsrg_sg)
coefficient_interval_plot3(rsrg_sg)
try:
    coefficient_interval_plot4(rsrg_sg)
except:
    plt.show()
coefficient_interval_plot5(rsrg_sg)

###### ER

In [ ]:
random_sg = []
for graph in randoms:
    random_sg.append(get_spectral_gap(graph))

get_details(random_sg)
coefficient_interval_plot(random_sg)
coefficient_interval_plot2(random_sg)
coefficient_interval_plot3(random_sg)
try:
    coefficient_interval_plot4(random_sg)
except:
    plt.show()
coefficient_interval_plot5(random_sg)

### Compare

In [ ]:
datas = [rsrbg_sg, rsrg_sg, random_sg]
labels = ['rsrbg', 'rsrg', 'er']
x_label = 'graph number'
y_label = 'spectral gap'
title = 'comparing spectral gap of graphs'
compare_datas(datas, labels, x_label=x_label, y_label=y_label, title=title)

# Centrality Of Global Subgraph

###### RSRBG

In [ ]:
rsrbg_cgs = []
for graph in rsrbgs:
    rsrbg_cgs.append(get_centrality_of_global_subgraph(graph))

get_details(rsrbg_cgs)
coefficient_interval_plot(rsrbg_cgs)
coefficient_interval_plot2(rsrbg_cgs)
coefficient_interval_plot3(rsrbg_cgs)
coefficient_interval_plot4(rsrbg_cgs)
coefficient_interval_plot5(rsrbg_cgs)

###### RSRG

In [ ]:
rsrg_cgs = []
for graph in rsrgs:
    rsrg_cgs.append(get_centrality_of_global_subgraph(graph))

get_details(rsrg_cgs)
coefficient_interval_plot(rsrg_cgs)
coefficient_interval_plot2(rsrg_cgs)
coefficient_interval_plot3(rsrg_cgs)
coefficient_interval_plot4(rsrg_cgs)
coefficient_interval_plot5(rsrg_cgs)

###### ER

In [ ]:
random_cgs = []
for graph in randoms:
    random_cgs.append(get_centrality_of_global_subgraph(graph))

get_details(random_cgs)
coefficient_interval_plot(random_cgs)
coefficient_interval_plot2(random_cgs)
coefficient_interval_plot3(random_cgs)
coefficient_interval_plot4(random_cgs)
coefficient_interval_plot5(random_cgs)

### Compare

In [ ]:
datas = [rsrbg_cgs, rsrg_cgs, random_cgs]
labels = ['rsrbg', 'rsrg', 'er']
x_label = 'graph number'
y_label = 'centrality of global subgraph'
title = 'comparing centrality of global subgraph of graphs'
compare_datas(datas, labels, x_label=x_label, y_label=y_label, title=title)

# Average Eigvals

###### RSRBG

In [ ]:
rsrbg_ae = []
for graph in rsrbgs:
    rsrbg_ae.append(get_average_eigvals(graph))
    
get_details(rsrbg_ae)
coefficient_interval_plot(rsrbg_ae)
coefficient_interval_plot2(rsrbg_ae)
coefficient_interval_plot3(rsrbg_ae)
coefficient_interval_plot4(rsrbg_ae)
coefficient_interval_plot5(rsrbg_ae)

###### RSRG

In [ ]:
rsrg_ae = []
for graph in rsrgs:
    rsrg_ae.append(get_average_eigvals(graph))

get_details(rsrg_ae)
coefficient_interval_plot(rsrg_ae)
coefficient_interval_plot2(rsrg_ae)
coefficient_interval_plot3(rsrg_ae)
coefficient_interval_plot4(rsrg_ae)
coefficient_interval_plot5(rsrg_ae)

###### ER

In [ ]:
random_ae = []
for graph in randoms:
    random_ae.append(get_average_eigvals(graph))

get_details(random_ae)
coefficient_interval_plot(random_ae)
coefficient_interval_plot2(random_ae)
coefficient_interval_plot3(random_ae)
coefficient_interval_plot4(random_ae)
coefficient_interval_plot5(random_ae)

### Compare

In [ ]:
datas = [rsrbg_ae, rsrg_ae, random_ae]
labels = ['rsrbg', 'rsrg', 'er']
x_label = 'graph number'
y_label = 'average eigvals'
title = 'comparing average eigvals of graphs'
compare_datas(datas, labels, x_label=x_label, y_label=y_label, title=title)

# Node Similarity

###### RSRBG

In [ ]:
rsrbg_ns = []
for graph in rsrbgs:
    rsrbg_ns.append(is_node_similar(graph))

rsrbg_node_similar = (sum(rsrbg_ns) / len(rsrbg_ns)) * 100
print(f'node similar: {rsrbg_node_similar}%')

###### RSRG

In [ ]:
rsrg_ns = []
for graph in rsrgs:
    rsrg_ns.append(is_node_similar(graph))

rsrg_node_similar = (sum(rsrg_ns) / len(rsrg_ns)) * 100
print(f'node similar: {rsrg_node_similar}%')

###### ER

In [ ]:
random_ns = []
for graph in randoms:
    random_ns.append(is_node_similar(graph))

random_node_similar = (sum(random_ns) / len(random_ns)) * 100
print(f'node similar: {random_node_similar}%')

# Symmetry

###### RSRBG

In [ ]:
rsrbg_symmetry = []
for graph in rsrbgs:
    rsrbg_symmetry.append(is_symmetry(graph))


rsrbg_sym = (sum(rsrbg_symmetry) / len(rsrbg_symmetry)) * 100
print(f'symmetry: {rsrbg_sym}%')

###### RSRG

In [ ]:
rsrg_symmetry = []
for graph in rsrgs:
    rsrg_symmetry.append(is_symmetry(graph))

rsrg_sym = (sum(rsrg_symmetry) / len(rsrg_symmetry)) * 100
print(f'symmetry: {rsrg_sym}%')

###### ER

In [ ]:
random_symmetry = []
for graph in randoms:
    random_symmetry.append(is_symmetry(graph))

random_sym = (sum(random_symmetry) / len(random_symmetry)) * 100
print(f'symmetry: {random_sym}%')

# Laplacian Energy

###### RSRBG

In [ ]:
rsrbg_le = []
for graph in rsrbgs:
    try:
        rsrbg_le.append(get_laplacian_energy(graph))
    except:
        rsrbg_le.append(0) # in case graph is not connected

get_details(rsrbg_le)
coefficient_interval_plot(rsrbg_le)
coefficient_interval_plot2(rsrbg_le)
coefficient_interval_plot3(rsrbg_le)
coefficient_interval_plot4(rsrbg_le)
coefficient_interval_plot5(rsrbg_le)

###### RSRG

In [ ]:
rsrg_le = []
for graph in rsrgs:
    try:
        rsrg_le.append(get_laplacian_energy(graph))
    except:
        rsrg_le.append(0) # in case graph is not connected

get_details(rsrg_le)
coefficient_interval_plot(rsrg_le)
coefficient_interval_plot2(rsrg_le)
coefficient_interval_plot3(rsrg_le)
coefficient_interval_plot4(rsrg_le)
coefficient_interval_plot5(rsrg_le)

###### ER

In [ ]:
random_le = []
for graph in randoms:
    try:
        random_le.append(get_laplacian_energy(graph))
    except:
        random_le.append(0) # in case graph is not connected

get_details(random_le)
coefficient_interval_plot(random_le)
coefficient_interval_plot2(random_le)
coefficient_interval_plot3(random_le)
coefficient_interval_plot4(random_le)
coefficient_interval_plot5(random_le)

### Compare

In [ ]:
datas = [rsrbg_le, rsrg_le, random_le]
labels = ['rsrbg', 'rsrg', 'er']
x_label = 'graph number'
y_label = 'laplacian energy'
title = 'comparing laplacian energy of graphs'
compare_datas(datas, labels, x_label=x_label, y_label=y_label, title=title)

# 

# Monte-Carlo Simulation

# Initial Parameters

In [ ]:
MC_NODE_NUMBER = 100
# rsrg parameters:
MC_UPPER_COPY_NUMBER_D1 = 15
MC_LOWER_COPY_NUMBER_D1 = 10
MC_LOWER_RSRG_P = 0.125
MC_LOWER_AVERAGE_D_RSRG = 2
MC_UPPER_AVERAGE_D_RSRG = 5
MC_RSRG_PROBABILITIES = [1, 0.75, 0.5, 0.25, 0.125]

MC_GRAPH_NUMBER = 100
MC_SIMULATION_NUMBER = 100
COLOR = ['red', 'green', 'blue', 'gray', 'yellow', 'brown', 'black']
plt.figure(figsize = (8,8)) #Specify the size of figure

# Making RSRG

In [ ]:
def make_rsrg_graph(graph_no):
    seed_values = random.sample(range(1, 34098),GRAPH_NUMBER)

    rsrgs = []
    probability_of_rsrg = []
    d1_of_rsrg = []
    d2_of_rsrg = []
    for i in range(graph_no):
        average_d = 0
        p = 0
        d1 = 0
        d2 = 0
        n = 0
        while average_d > MC_UPPER_AVERAGE_D_RSRG or average_d == 0:
            p = random.choice(MC_RSRG_PROBABILITIES)
            d1 = random.randint(LOWER_COPY_NUMBER_D1, UPPER_COPY_NUMBER_D1)
            d2 = random.randint(LOWER_COPY_NUMBER_D1, UPPER_COPY_NUMBER_D1)
            n = NODE_NUMBER
            average_d = (d1 * (1 - p) + p * d2) * p
        
        graph = generate_rsrg(p, d1, d2, n, seed=seed_values[i])
        rsrgs.append(graph)
        actuan_average_d = (2*graph.number_of_edges()) / float(graph.number_of_nodes())
        print(f'p: {p}, d1: {d1}, d2: {d2}, technical <d>: {average_d}, <d>: {actuan_average_d}')
        probability_of_rsrg.append(p)
        d1_of_rsrg.append(d1)
        d2_of_rsrg.append(d2)
    
    return rsrgs

In [ ]:
def simulate():
        rsrgs = make_rsrg_graph(GRAPH_NUMBER)
        rsrg_ac = [get_algebraic_connectivity(graph) for graph in rsrgs]
        get_details(rsrg_ac)
        coefficient_interval_plot(rsrg_ac)
        coefficient_interval_plot2(rsrg_ac)
        coefficient_interval_plot3(rsrg_ac)
        coefficient_interval_plot4(rsrg_ac)
        coefficient_interval_plot5(rsrg_ac)
        
        draw_pdf(rsrg_ac, 'algebraic connectivity pdf')
        draw_pdf2(rsrg_ac, 'algebraic connectivity pdf')
        draw_pdf3(rsrg_ac, 'algebraic connectivity pdf')

In [ ]:
for i in range(SIMULATION_NUMBER):
    simulate()